# Evaluation

In [2]:
import os
import logging
import subprocess
from pathlib import Path

## Setup

### Store Directory

In [20]:
storedir = None  # Set this to persist evaluation results/checkpoints

In [21]:
if storedir is not None:
    checkpoint_storedir = f"{storedir}/checkpoints"
    Path(checkpoint_storedir).mkdir(exist_ok=True)

    data_storedir = f"{storedir}/data"
    Path(data_storedir).mkdir(exist_ok=True)
else:
    checkpoint_storedir = None
    data_storedir = None
    
try:
    job_id = os.environ['PBS_JOBID'].split('.pbs')[0]
except KeyError:
    job_id = 'local'

In [22]:
logging.basicConfig()
logger = logging.getLogger('job')
logger.setLevel(logging.INFO)

### Imports

In [ ]:
logger.info('Importing third-party packages ...')

import torch
from torch.utils.data import DataLoader, Subset, ConcatDataset, random_split
from tqdm import tqdm

from volatility_smoothing.utils.chunk import chunked
from volatility_smoothing.utils.train.loss import Loss
from volatility_smoothing.utils.options_data import SPXOptionsDataset
from volatility_smoothing.utils.train.dataset import GNOOptionsDataset
from volatility_smoothing.utils.chunk import chunked
from volatility_smoothing.utils.train import misc

### Device

In [ ]:
logger.info(f"Defining device (torch.cuda.is_available()={torch.cuda.is_available()})")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

logger.info(f'Running using device `{device}`')

if device.type == 'cuda':
    result = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE)
    formatted_result = str(result.stdout).replace('\\n', '\n').replace('\\t', '\t')##

    logger.info(formatted_result)
    logger.info(f'Device count: {torch.cuda.device_count()}')
    logger.info(f'Visible devices count: {os.environ["CUDA_VISIBLE_DEVICES"]}')

## Datasets

In [25]:
os.environ['OPDS_CACHE_DIR'] = os.path.expanduser('~/.cache/opds')  # directory where to place the processed files
os.environ['OPDS_WRDS_DATA_DIR'] = os.path.abspath("../data/wrds/spx")  # <- .csv file from WRDS should be place inside this directory
dataset = SPXOptionsDataset()

In [26]:
from datetime import datetime
from volatility_smoothing.utils.options_data import OptionsDataset


def split_dataset(dataset: OptionsDataset):
    """Splits dataset in <2021 training portion, and 2020 test portion (sub-split into months)"""
    train_indices = []
    val_indices = [[] for _ in range(12)]

    for idx, file_path in enumerate(dataset.file_paths):
        date_str = str(file_path).split('_')[-1].replace('.pt', '')
        date = datetime.strptime(date_str, '%Y-%m-%d-%H-%M-%S')

        if date.year < 2021:
            train_indices.append(idx)
        elif date.year == 2021:
            month = date.month - 1
            val_indices[month].append(idx)

    train_dataset, dev_dataset = random_split(Subset(GNOOptionsDataset(dataset), train_indices), [0.982, 0.018])
    test_datasets = [Subset(GNOOptionsDataset(dataset), indices) for indices in val_indices]

    return train_dataset, dev_dataset, test_datasets

In [27]:
train_dataset, dev_dataset, test_datasets = split_dataset(dataset)

## Model and Optimizer

In [28]:
lr = 1e-4
weight_decay = 1e-5

checkpoint_path = "../train/store/9448705/checkpoints/checkpoint_final.pt"
model, _ = misc.load_checkpoint(checkpoint_path, device=device)
model.to(device);
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay) 

## Evaluation/Finetuning Hyperparameters

In [29]:
num_workers = 4

epochs = 10  # Finetune epochs, set to 0 to skip and just evaluate
batch_size = 32  # Finetune batch size, will be augmented by same amount of training data

# mesh sizes on which to evaluate arbitrage metrics
step_r = 0.02
step_z = 0.01

In [30]:
train_loss = Loss()
dev_loss = Loss(step_r=step_r, step_z=step_z)

## Evaluation

In [31]:
def finetune(model, optimizer, train_dataset: GNOOptionsDataset, finetune_dataset: GNOOptionsDataset, dev_dataset: GNOOptionsDataset, **kwargs):

    logger = logging.getLogger('job')

    kwargs = kwargs.copy()
    num_workers = kwargs.pop('num_workers', 0)
    epochs = kwargs.pop('epochs', 10)
    batch_size = kwargs.pop('batch_size', 64)
    dev_loss = kwargs.pop('dev_loss', Loss())
    callback = kwargs.pop('callback', lambda sample_loss: sample_loss.backward())
    checkpoint_storedir = kwargs.pop('checkpoint_storedir', None)

    loss = Loss(**kwargs)
    train_dataloader = DataLoader(train_dataset, batch_size=1, collate_fn=loss.collate_fn, shuffle=True, num_workers=num_workers, pin_memory=False)
    finetune_dataloader = DataLoader(finetune_dataset, batch_size=1, collate_fn=train_loss.collate_fn, shuffle=True, num_workers=num_workers, pin_memory=False)

    logger.info(50 * "=")
    logger.info("Training start:")
    logger.info(f"Epochs: {epochs}")
    logger.info(loss)
    logger.info(50 * "=")

    for epoch in range(epochs):
        model.train()

        train_iterator = iter(train_dataloader)
        finetune_iterator = iter(finetune_dataloader)

        for batch_idx in (iterations := tqdm(chunked(list(range(len(finetune_dataloader))), batch_size))):
            batch = ([next(train_iterator) for _ in batch_idx]
                     + [next(finetune_iterator) for _ in batch_idx])
            
            optimizer.zero_grad()
            batch_loss, loss_infos = loss.compute_batch_loss(model, batch, callback, device)
            loss_str = loss.format_loss_str(loss_infos)                                        
            iterations.set_description(loss_str)
            optimizer.step()
        
            if (iterations.n % 10 == 0) and (storedir is not None):
                logger.info(f"Epoch {epoch}; {iterations.n}/{len(iterations)} -- {loss_str}")


        # Dev loss
        df_val, df_rel, df_fit = dev_loss.evaluate(model, dev_dataset, device=device, num_workers=num_workers)
        logger.info(f"Epoch {epoch} Dev: {df_val.describe()}")

        # Checkpointing
        if checkpoint_storedir is not None and not batch_loss.isnan():
            checkpoint = {
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
            }
            torch.save(checkpoint, f"{checkpoint_storedir}/{job_id}_checkpoint_{epoch}.pt")

    return model

In [ ]:
torch.cuda.empty_cache()

logger.info(50 * "=")
logger.info(f"Evaluation start (Retraining epochs: {epochs}).")
logger.info(50 * "=")

finetune_dataset = Subset(train_dataset, [])
try:
    for k, test_dataset in enumerate(test_datasets):

        df_val, df_rel, df_fit = dev_loss.evaluate(model, test_dataset, device=device, num_workers=num_workers, storedir=storedir, logger=logger)

        finetune_dataset = ConcatDataset([finetune_dataset, test_dataset])
        finetune(model, optimizer, train_dataset, finetune_dataset, dev_dataset,
                 dev_loss=dev_loss, epochs=epochs, batch_size=batch_size, num_workers=num_workers, checkpoint_storedir=checkpoint_storedir)

except KeyboardInterrupt:
    logging.info("Training aborted")
else:
    logging.info("Training complete")
finally:
    if checkpoint_storedir is not None:
        checkpoint = {
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }
        torch.save(model, f"{checkpoint_storedir}/checkpoint_final.pt")
    model.eval()